## Data Validator
Notebook that reads in old scraped job advertisement data and checks for missing information.

If there is missing informtion, this notebook has the functionality to fill it in.

In [30]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
from lxml import etree

In [31]:
data_file_path = r"C:\Users\ethan_haque\Desktop\repos\princeton-job-advertisement-scraper\data\2021-12-19-chronicles_of_higher_ed.csv"

In [32]:
csv_data = pd.read_csv(data_file_path, index_col=0)
csv_data.columns.values

array(['Job Title', 'Job URL', 'Diversity Job', 'Employer', 'Location',
       'Salary', 'Date Posted', 'Description', 'Position Type 0.0',
       'Position Type 0.1', 'Position Type 0.2', 'Position Type 0.3',
       'Position Type 0.4', 'Position Type 0.5', 'Position Type 0.6',
       'Position Type 0.7', 'Position Type 0.8', 'Position Type 0.9',
       'Position Type 1.0'], dtype=object)

In [33]:
def get_urls_missing_date_posted(df):
    no_dates = df[df["Date Posted"].isna()]
    urls = no_dates.loc[:,"Job URL"]
    return urls

# get_urls_missing_date_posted(csv_data)

In [45]:
def get_date_posted(url):
    request = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(request.content, "html.parser")
    dom = etree.HTML(str(soup))
    date_xpath = '//*[@id="main"]/div/div[3]/div[1]/div[1]/div[1]/div/div[1]/dl/dd[4]'
    date_element = dom.xpath(date_xpath)
    return date_element[0].text
    
get_date_posted("https://jobs.chronicle.com/job/37302598/assistant-professor-in-chemistry-open-area-/")
    

Aug 11, 2022
